In [ ]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd  /content/drive/'My Drive'/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
!pip install rasterio

import rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.3 MB 354 kB/s 


In [ ]:
true_img = rasterio.open('GEE_Dynamic_2016_3_class_value_image/9_9.png', "r").read()

predicted_img = rasterio.open('dfc2021-msd-baseline-master/results/unet_both_baseline_nicfi_main/output/99_2016.tif',"r").read()

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


## UNet - Both

In [ ]:
val_list = ['9_9','0_15','14_12','0_1','2_14','6_8','1_14','0_9','1_4','1_6','1_7','1_9','10_0','10_4','10_6','10_8','11_15','11_2','13_13','13_8','14_8','15_0','15_11','15_8','15_9','2_3','3_0','3_3','3_6','3_6','4_11','4_7','4_9','5_10','5_13','5_14','5_2','5_4','6_11','6_13','6_3','7_0','7_10','7_2','8_13','8_3','9_1','9_2','9_7','7_12','7_14']


img_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/' ##00.tif
img_2019 = '/content/drive/MyDrive/GEE_Dynamic_2019_3_class_value_image/' #00.tif
#label_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/'
output = '/content/drive/MyDrive/dfc2021-msd-baseline-master/results/unet_both_baseline_nicfi_main/output/'

image_fn = []
label_fn = []

for i in val_list:
  x,y = i.split('_')
  image_fn.append(img_2016 + i + '.png')
  label_fn.append(output + x + y + '_2016.tif')
  image_fn.append(img_2019 + i + '.png')
  label_fn.append(output + x + y + '_2019.tif')

dynamic_world_labels = image_fn
predicted_labels = label_fn

In [ ]:
precision_list_land = []
recall_list_land = []
iou_score_land = []

precision_list_trees = []
recall_list_trees = []
iou_score_trees = []

precision_list_water = []
recall_list_water = []
iou_score_water = []

precision_list_weighted = []
recall_list_weighted = []

accuracy_overall = []
iou_overall = []

for i in range(len(dynamic_world_labels)):
    true_img = rasterio.open(dynamic_world_labels[i], "r").read()
    predicted_img = rasterio.open(predicted_labels[i],"r").read()
    if(true_img.shape != predicted_img.shape):
      continue;
    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
    m.update_state(true_img,predicted_img)
    iou_score_land.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
    m.update_state(true_img,predicted_img)
    iou_score_water.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
    m.update_state(true_img,predicted_img)
    iou_score_trees.append(m.result().numpy())

    '''l,b,h = true_img.shape
    true_img_vect = true_img.reshape(l*b*h)
    predicted_img_vect = predicted_img.reshape(l*b*h)

    report = classification_report(true_img_vect, predicted_img_vect, output_dict=True)
    #print(report)
    try:
      precision_list_land.append(report['2']['precision'])
    except:
      pass
    try:
      precision_list_trees.append(report['1']['precision'])
    except:
      pass
    try:
      precision_list_water.append(report['0']['precision'])
    except:
      pass


    try:
      recall_list_land.append(report['2']['recall'])
    except:
      pass
    try:
      recall_list_trees.append(report['1']['recall'])
    except:
      pass
    try:
      recall_list_water.append(report['0']['recall'])
    except:
      pass
    
    accuracy_overall.append(report['accuracy'])

'''

In [ ]:
from statistics import mean
'''print("Average Impervios Precision:", mean(precision_list_land)*100)
print("Average Impervios Recall:", mean(recall_list_land)*100)
print("Average Tree Canopy Precision:", mean(precision_list_trees)*100)
print("Average Tree Canopy Recall:", mean(recall_list_trees)*100)
print("Average Water Precision:", mean(precision_list_water)*100)
print("Average Water Recall:", mean(recall_list_water)*100)
print("Average Accuracy:", mean(accuracy_overall)*100)'''
print("Average IoU Trees:", mean(iou_score_trees)*100)
print("Average IoU Land:", mean(iou_score_land)*100)
print("Average IoU Water:", mean(iou_score_water)*100)
#print("Average IoU Overall:", mean(iou_overall)*100)

Average IoU Trees: 84.77376103401184
Average IoU Land: 53.90557646751404
Average IoU Water: 0.0


## FCN Both

In [ ]:
val_list = ['9_9','0_15','14_12','0_1','2_14','6_8','1_14','0_9','1_4','1_6','1_7','1_9','10_0','10_4','10_6','10_8','11_15','11_2','13_13','13_8','14_8','15_0','15_11','15_8','15_9','2_3','3_0','3_3','3_6','3_6','4_11','4_7','4_9','5_10','5_13','5_14','5_2','5_4','6_11','6_13','6_3','7_0','7_10','7_2','8_13','8_3','9_1','9_2','9_7','7_12','7_14']


img_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/' ##00.tif
img_2019 = '/content/drive/MyDrive/GEE_Dynamic_2019_3_class_value_image/' #00.tif
#label_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/'
output = '/content/drive/MyDrive/dfc2021-msd-baseline-master/results/fcn_both_baseline_nicfi_main/output/'

image_fn = []
label_fn = []

for i in val_list:
  x,y = i.split('_')
  image_fn.append(img_2016 + i + '.png')
  label_fn.append(output + x + y + '_2016.tif')
  image_fn.append(img_2019 + i + '.png')
  label_fn.append(output + x + y + '_2019.tif')

dynamic_world_labels = image_fn
predicted_labels = label_fn

In [ ]:
precision_list_land = []
recall_list_land = []
iou_score_land = []

precision_list_trees = []
recall_list_trees = []
iou_score_trees = []

precision_list_water = []
recall_list_water = []
iou_score_water = []

precision_list_weighted = []
recall_list_weighted = []

accuracy_overall = []
iou_overall = []

for i in range(len(dynamic_world_labels)):
    true_img = rasterio.open(dynamic_world_labels[i], "r").read()
    predicted_img = rasterio.open(predicted_labels[i],"r").read()
    if(true_img.shape != predicted_img.shape):
      continue;
    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
    m.update_state(true_img,predicted_img)
    iou_score_land.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
    m.update_state(true_img,predicted_img)
    iou_score_water.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
    m.update_state(true_img,predicted_img)
    iou_score_trees.append(m.result().numpy())

    '''l,b,h = true_img.shape
    true_img_vect = true_img.reshape(l*b*h)
    predicted_img_vect = predicted_img.reshape(l*b*h)

    report = classification_report(true_img_vect, predicted_img_vect, output_dict=True)
    #print(report)
    try:
      precision_list_land.append(report['2']['precision'])
    except:
      pass
    try:
      precision_list_trees.append(report['1']['precision'])
    except:
      pass
    try:
      precision_list_water.append(report['0']['precision'])
    except:
      pass


    try:
      recall_list_land.append(report['2']['recall'])
    except:
      pass
    try:
      recall_list_trees.append(report['1']['recall'])
    except:
      pass
    try:
      recall_list_water.append(report['0']['recall'])
    except:
      pass
    
    accuracy_overall.append(report['accuracy'])

'''

In [ ]:
from statistics import mean
'''print("Average Impervios Precision:", mean(precision_list_land)*100)
print("Average Impervios Recall:", mean(recall_list_land)*100)
print("Average Tree Canopy Precision:", mean(precision_list_trees)*100)
print("Average Tree Canopy Recall:", mean(recall_list_trees)*100)
print("Average Water Precision:", mean(precision_list_water)*100)
print("Average Water Recall:", mean(recall_list_water)*100)
print("Average Accuracy:", mean(accuracy_overall)*100)'''
print("Average IoU Trees:", mean(iou_score_trees)*100)
print("Average IoU Land:", mean(iou_score_land)*100)
print("Average IoU Water:", mean(iou_score_water)*100)
#print("Average IoU Overall:", mean(iou_overall)*100)

Average IoU Trees: 78.1981885433197
Average IoU Land: 15.341858565807343
Average IoU Water: 0.0


## UNet - Water

In [ ]:
val_list = ['9_9','0_15','14_12','0_1','2_14','6_8','1_14','0_9','1_4','1_6','1_7','1_9','10_0','10_4','10_6','10_8','11_15','11_2','13_13','13_8','14_8','15_0','15_11','15_8','15_9','2_3','3_0','3_3','3_6','3_6','4_11','4_7','4_9','5_10','5_13','5_14','5_2','5_4','6_11','6_13','6_3','7_0','7_10','7_2','8_13','8_3','9_1','9_2','9_7','7_12','7_14']


img_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/' ##00.tif
img_2019 = '/content/drive/MyDrive/GEE_Dynamic_2019_3_class_value_image/' #00.tif
#label_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/'
output = '/content/drive/MyDrive/dfc2021-msd-baseline-master/results/unet_both_baseline_nicfi_water/output/'

image_fn = []
label_fn = []

for i in val_list:
  x,y = i.split('_')
  image_fn.append(img_2016 + i + '.png')
  label_fn.append(output + x + y + '_2016.tif')
  image_fn.append(img_2019 + i + '.png')
  label_fn.append(output + x + y + '_2019.tif')

dynamic_world_labels = image_fn
predicted_labels = label_fn

In [ ]:
precision_list_land = []
recall_list_land = []
iou_score_land = []

precision_list_trees = []
recall_list_trees = []
iou_score_trees = []

precision_list_water = []
recall_list_water = []
iou_score_water = []

precision_list_weighted = []
recall_list_weighted = []

accuracy_overall = []
iou_overall = []

for i in range(len(dynamic_world_labels)):
    true_img = rasterio.open(dynamic_world_labels[i], "r").read()
    predicted_img = rasterio.open(predicted_labels[i],"r").read()
    if(true_img.shape != predicted_img.shape):
      continue;
    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
    m.update_state(true_img,predicted_img)
    iou_score_land.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
    m.update_state(true_img,predicted_img)
    iou_score_water.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
    m.update_state(true_img,predicted_img)
    iou_score_trees.append(m.result().numpy())

    '''l,b,h = true_img.shape
    true_img_vect = true_img.reshape(l*b*h)
    predicted_img_vect = predicted_img.reshape(l*b*h)

    report = classification_report(true_img_vect, predicted_img_vect, output_dict=True)
    #print(report)
    try:
      precision_list_land.append(report['2']['precision'])
    except:
      pass
    try:
      precision_list_trees.append(report['1']['precision'])
    except:
      pass
    try:
      precision_list_water.append(report['0']['precision'])
    except:
      pass


    try:
      recall_list_land.append(report['2']['recall'])
    except:
      pass
    try:
      recall_list_trees.append(report['1']['recall'])
    except:
      pass
    try:
      recall_list_water.append(report['0']['recall'])
    except:
      pass
    
    accuracy_overall.append(report['accuracy'])

'''

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [ ]:
from statistics import mean
'''print("Average Impervios Precision:", mean(precision_list_land)*100)
print("Average Impervios Recall:", mean(recall_list_land)*100)
print("Average Tree Canopy Precision:", mean(precision_list_trees)*100)
print("Average Tree Canopy Recall:", mean(recall_list_trees)*100)
print("Average Water Precision:", mean(precision_list_water)*100)
print("Average Water Recall:", mean(recall_list_water)*100)
print("Average Accuracy:", mean(accuracy_overall)*100)'''
print("Average IoU Trees:", mean(iou_score_trees)*100)
print("Average IoU Land:", mean(iou_score_land)*100)
print("Average IoU Water:", mean(iou_score_water)*100)
#print("Average IoU Overall:", mean(iou_overall)*100)

Average IoU Trees: 79.88708019256592
Average IoU Land: 50.50587058067322
Average IoU Water: 0.0


In [ ]:
precision_list_land = []
recall_list_land = []
iou_score_land = []

precision_list_trees = []
recall_list_trees = []
iou_score_trees = []

precision_list_water = []
recall_list_water = []
iou_score_water = []

precision_list_weighted = []
recall_list_weighted = []

accuracy_overall = []
iou_overall = []

for i in range(len(dynamic_world_labels)):
    true_img = rasterio.open(dynamic_world_labels[i], "r").read()
    predicted_img = rasterio.open(predicted_labels[i],"r").read()
    if(true_img.shape != predicted_img.shape):
      continue;
    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
    m.update_state(true_img,predicted_img)
    iou_score_land.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
    m.update_state(true_img,predicted_img)
    iou_score_water.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
    m.update_state(true_img,predicted_img)
    iou_score_trees.append(m.result().numpy())

    l,b,h = true_img.shape
    true_img_vect = true_img.reshape(l*b*h)
    predicted_img_vect = predicted_img.reshape(l*b*h)

    report = classification_report(true_img_vect, predicted_img_vect, output_dict=True)
    #print(report)
    try:
      precision_list_land.append(report['2']['precision'])
    except:
      pass
    try:
      precision_list_trees.append(report['1']['precision'])
    except:
      pass
    try:
      precision_list_water.append(report['0']['precision'])
    except:
      pass


    try:
      recall_list_land.append(report['2']['recall'])
    except:
      pass
    try:
      recall_list_trees.append(report['1']['recall'])
    except:
      pass
    try:
      recall_list_water.append(report['0']['recall'])
    except:
      pass
    
    accuracy_overall.append(report['accuracy'])



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: No

In [ ]:
from statistics import mean
print("Average Impervios Precision:", mean(precision_list_land)*100)
print("Average Impervios Recall:", mean(recall_list_land)*100)
print("Average Tree Canopy Precision:", mean(precision_list_trees)*100)
print("Average Tree Canopy Recall:", mean(recall_list_trees)*100)
print("Average Water Precision:", mean(precision_list_water)*100)
print("Average Water Recall:", mean(recall_list_water)*100)
print("Average Accuracy:", mean(accuracy_overall)*100)
print("Average IoU Trees:", mean(iou_score_trees)*100)
print("Average IoU Land:", mean(iou_score_land)*100)
print("Average IoU Water:", mean(iou_score_water)*100)
print("Average IoU Overall:", mean(iou_overall)*100)

Average Impervios Precision: 64.63573421733523
Average Impervios Recall: 18.665487768030463
Average Tree Canopy Precision: 79.71772142540368
Average Tree Canopy Recall: 97.73574926925667
Average Water Precision: 0.0
Average Water Recall: 0.0
Average Accuracy: 79.81613049393485
Average IoU Trees: 78.1981885433197
Average IoU Land: 15.341858565807343
Average IoU Water: 0.0


StatisticsError: ignored

In [ ]:
l,b,h = true_img.shape
mask = tf.reshape(true_img, (l*b*h))
pred_mask = tf.reshape(predicted_img, (l*b*h))

In [ ]:
mask = true_img.reshape(l*b*h)
pred_mask = predicted_img.reshape(l*b*h)

In [ ]:
import tensorflow as tf
from sklearn.metrics import classification_report

#mask = tf.reshape(true_img, (h*w))
#pred_mask = tf.reshape(predicted_img, (h*w))
m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0,1,2])
m.update_state(true_img,predicted_img)
print(m.result().numpy())

m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
m.update_state(true_img,predicted_img)
print(m.result().numpy())

m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
m.update_state(true_img,predicted_img)
print(m.result().numpy())

m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
m.update_state(true_img,predicted_img)
print(m.result().numpy())


report = classification_report(mask,pred_mask, output_dict=True)
#print(report)
try:
  print(report['2']['precision'])
except:
  pass
try:
  print(report['1']['precision'])
except:
  pass
try:
  print(report['0']['precision'])
except:
  pass


try:
  print(report['2']['recall'])
except:
  pass
try:
  print(report['1']['recall'])
except:
  pass
try:
  print(report['0']['recall'])
except:
  pass

print(report['accuracy'])

0.4567909
0.0
0.6988587
0.671514


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.7366462185339392
0.9147071969565546
0.0
0.9316185127509979
0.7163702190826398
0.0
0.811271746014785


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
i = 0
precision_list_land = []
recall_list_land = []
iou_score_land = []

precision_list_trees = []
recall_list_trees = []
iou_score_trees = []

precision_list_water = []
recall_list_water = []
iou_score_water = []

precision_list_weighted = []
recall_list_weighted = []

accuracy_overall = []
iou_overall = []

for image, mask in new_train_ds.take(4):
  pred_mask = model.predict(image)
  pred_mask = tf.math.argmax(pred_mask, axis=-1)
  mask = tf.reshape(mask, (32,224*224))
  pred_mask = tf.reshape(pred_mask, (32,224*224))
  for j in range(32):
    mask_im = np.array(mask[j], dtype = 'uint8')
    pred_mask_im = np.array(pred_mask[j])

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[2])
    m.update_state(mask_im,pred_mask_im)
    iou_score_land.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0])
    m.update_state(mask_im,pred_mask_im)
    iou_score_water.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[1])
    m.update_state(mask_im,pred_mask_im)
    iou_score_trees.append(m.result().numpy())

    m = tf.keras.metrics.IoU(num_classes=3, target_class_ids=[0,1,2])
    m.update_state(mask_im,pred_mask_im)
    iou_overall.append(m.result().numpy())

    report = classification_report(mask_im, pred_mask_im, output_dict=True)
    #print(report)
    try:
      precision_list_land.append(report['2']['precision'])
    except:
      pass
    try:
      precision_list_trees.append(report['1']['precision'])
    except:
      pass
    try:
      precision_list_water.append(report['0']['precision'])
    except:
      pass


    try:
      recall_list_land.append(report['2']['recall'])
    except:
      pass
    try:
      recall_list_trees.append(report['1']['recall'])
    except:
      pass
    try:
      recall_list_water.append(report['0']['recall'])
    except:
      pass
    
    accuracy_overall.append(report['accuracy'])

In [ ]:
from statistics import mean
print("Average Impervios Precision:", mean(precision_list_land)*100)
print("Average Impervios Recall:", mean(recall_list_land)*100)
print("Average Tree Canopy Precision:", mean(precision_list_trees)*100)
print("Average Tree Canopy Recall:", mean(recall_list_trees)*100)
print("Average Water Precision:", mean(precision_list_water)*100)
print("Average Water Recall:", mean(recall_list_water)*100)
print("Average Accuracy:", mean(accuracy_overall)*100)
print("Average IoU Trees:", mean(iou_score_trees)*100)
print("Average IoU Land:", mean(iou_score_land)*100)
print("Average IoU Water:", mean(iou_score_water)*100)
print("Average IoU Overall:", mean(iou_overall)*100)

In [ ]:
val_list = ['9_9','0_15','14_12','0_1','2_14','6_8','1_14','0_9','1_4','1_6','1_7','1_9','10_0','10_4','10_6','10_8','11_15','11_2','13_13','13_8','14_8','15_0','15_11','15_8','15_9','2_3','3_0','3_3','3_6','3_6','4_11','4_7','4_9','5_10','5_13','5_14','5_2','5_4','6_11','6_13','6_3','7_0','7_10','7_2','8_13','8_3','9_1','9_2','9_7','7_12','7_14']


img_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/' ##00.tif
img_2019 = '/content/drive/MyDrive/GEE_Dynamic_2019_3_class_value_image/' #00.tif
#label_2016 = '/content/drive/MyDrive/GEE_Dynamic_2016_3_class_value_image/'
output = '/content/drive/MyDrive/dfc2021-msd-baseline-master/results/unet_both_baseline_nicfi_main/output/'

image_fn = []
label_fn = []

for i in val_list:
  x,y = i.split('_')
  image_fn.append(img_2016 + i + '.png')
  label_fn.append(output + x + y + '_2016.tif')
  image_fn.append(img_2019 + i + '.png')
  label_fn.append(output + x + y + '_2019.tif')

dynamic_world_labels = image_fn
predicted_labels = label_fn